# Judging and Routing — Optimizing Resource Usage by Evaluating Problem Complexity

In the original Lab 2, we explored the **Orchestrator–Worker pattern**, where a planner sent the same question to multiple agents, and a judge assessed their responses to evaluate agent intelligence.

In this notebook, we extend that design by adding multiple judges and a routing component to optimize model usage based on task complexity. 

## Imports and Environment Setup

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
if openai_api_key and google_api_key and deepseek_api_key:
    print("All keys were loaded successfully")

In [ ]:
!ollama pull llama3.2
!ollama pull mistral

## Creating Models

The notebook uses instances of GPT, Gemini and DeepSeek APIs, along with two local models served via Ollama: ```llama3.2```  and ```mistral```.

In [4]:
model_specs = {
    "gpt-4o-mini" : None,
    "gemini-2.0-flash": {
        "api_key" : google_api_key,
        "url" : "https://generativelanguage.googleapis.com/v1beta/openai/"
    },
    "deepseek-chat" : {
        "api_key" : deepseek_api_key,
        "url" : "https://api.deepseek.com/v1"
    },
    "llama3.2" : {
        "api_key" : "ollama",
        "url" : "http://localhost:11434/v1"
    },
    "mistral" : {
        "api_key" : "ollama",
        "url" : "http://localhost:11434/v1"
    }
}

def create_model(model_name):
    spec = model_specs[model_name]
    if spec is None:
        return OpenAI()
    
    return OpenAI(api_key=spec["api_key"], base_url=spec["url"])

In [5]:
orchestrator_model = "gemini-2.0-flash"
generator = create_model(orchestrator_model)
router = create_model(orchestrator_model)

qa_models = {
    model_name : create_model(model_name) 
    for model_name in model_specs.keys()
}

judges = {
    model_name : create_model(model_name) 
    for model_name, specs in model_specs.items() 
    if not(specs) or specs["api_key"] != "ollama"
}

## Orchestrator-Worker Workflow

First, we generate a question to evaluate the intelligence of each LLM.

In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs "
request += "to evaluate and rank them based on their intelligence. " 
request += "Answer **only** with the question, no explanation or preamble."

messages = [{"role": "user", "content": request}]
messages

In [7]:
response = generator.chat.completions.create(
    model=orchestrator_model,
    messages=messages,
)
eval_question = response.choices[0].message.content

In [ ]:
display(Markdown(eval_question))

### Task Parallelization

Now, having the question and all the models instantiated it's time to see what each model has to say about the complex task it was given.

In [ ]:
question = [{"role": "user", "content": eval_question}]
answers = []
competitors = []

for name, model in qa_models.items():
    response = model.chat.completions.create(model=name, messages=question)
    answer = response.choices[0].message.content
    competitors.append(name)
    answers.append(answer)

answers

In [ ]:
report = "# Answer report for each of the 5 models\n\n"
report += "\n\n".join([f"## **Model: {model}**\n\n{answer}" for model, answer in zip(competitors, answers)])
display(Markdown(report))

### Synthetizer/Judge

The Judge Agents ranks the LLM responses based on coherence and relevance to the evaluation prompt. Judges vote and the final LLM ranking is based on the aggregated ranking of all three judges.

In [ ]:
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

together

In [12]:
judge_prompt = f"""
                You are judging a competition between {len(competitors)} LLM competitors.
                Each model has been given this nuanced question to evaluate their intelligence:

                {eval_question}

                Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
                Respond with JSON, and only JSON, with the following format:
                {{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}
                With 'best competitor number being ONLY the number', for instance:
                {{"results": ["5", "2", "4", ...]}}
                Here are the responses from each competitor:

                {together}

                Now respond with the JSON with the ranked order of the competitors, nothing else. Do NOT include MARKDOWN FORMATTING or CODE BLOCKS. ONLY the JSON
                """

judge_messages = [{"role": "user", "content": judge_prompt}]

In [ ]:
from collections import defaultdict
import re

N = len(competitors)
scores = defaultdict(int)
for judge_name, judge in judges.items():
    response = judge.chat.completions.create(
        model=judge_name,
        messages=judge_messages,
    )
    response = response.choices[0].message.content
    response_json = re.findall(r'\{.*?\}', response)[0]
    results = json.loads(response_json)["results"]
    ranks = [int(result) for result in results]
    print(f"Judge {judge_name} ranking:")
    for i, c in enumerate(ranks):
        model_name = competitors[c - 1]
        print(f"#{i+1} : {model_name}")
        scores[c - 1] += (N - i)
    print()

In [ ]:
sorted_indices = sorted(scores, key=scores.get)

# Convert to model names
ranked_model_names = [competitors[i] for i in sorted_indices]

print("Final ranking from best to worst:")
for i, name in enumerate(ranked_model_names[::-1], 1):
    print(f"#{i}: {name}")

## Routing Workflow

We now define a routing agent responsible for classifying task complexity and delegating the prompt to the most appropriate model.

In [15]:
def classify_question_complexity(question: str, routing_agent, routing_model) -> int:
    """
    Ask an LLM to classify the question complexity from 1 (easy) to 5 (very hard).
    """
    prompt = f"""
        You are a classifier responsible for assigning a complexity level to user questions, based on how difficult they would be for a language model to answer.

        Please read the question below and assign a complexity score from 1 to 5:

        - Level 1: Very simple factual or definitional question (e.g., “What is the capital of France?”)
        - Level 2: Slightly more involved, requiring basic reasoning or comparison
        - Level 3: Moderate complexity, requiring synthesis, context understanding, or multi-part answers
        - Level 4: High complexity, requiring abstract thinking, ethical judgment, or creative generation
        - Level 5: Extremely challenging, requiring deep reasoning, philosophical reflection, or long-term multi-step inference

        Respond ONLY with a single integer between 1 and 5 that best reflects the complexity of the question.

        Question:
        {question}
        """

    response = routing_agent.chat.completions.create(
        model=routing_model,
        messages=[{"role": "user", "content": prompt}]
    )
    try:
        return int(response.choices[0].message.content.strip())
    except Exception:
        return 3  # default to medium complexity on error
    
def route_question_to_model(question: str, models_by_rank, classifier_model=router, model_name=orchestrator_model):
    level = classify_question_complexity(question, classifier_model, model_name)
    selected_model_name = models_by_rank[level - 1]
    return selected_model_name

In [16]:
difficulty_prompts = [
    "Generate a very basic, factual question that a small or entry-level language model could answer easily. It should require no reasoning, just direct knowledge lookup.",
    "Generate a slightly involved question that requires basic reasoning, comparison, or combining two known facts. Still within the grasp of small models but not purely factual.",
    "Generate a moderately challenging question that requires some synthesis of ideas, multi-step reasoning, or contextual understanding. A mid-tier model should be able to answer it with effort.",
    "Generate a difficult question involving abstract thinking, open-ended reasoning, or ethical tradeoffs. The question should challenge large models to produce thoughtful and coherent responses.",
    "Generate an extremely complex and nuanced question that tests the limits of current language models. It should require deep reasoning, long-term planning, philosophy, or advanced multi-domain knowledge."
]
def generate_question(level, generator=generator, generator_model=orchestrator_model):
    prompt = (
        f"{difficulty_prompts[level - 1]}\n"
        "Answer only with the question, no explanation."
    )
    messages = [{"role": "user", "content": prompt}]
    response = generator.chat.completions.create(
        model=generator_model,  # or your planner model
        messages=messages
    )
    
    return response.choices[0].message.content



### Testing Routing Workflow

Finally, to test the routing workflow, we create a function that accepts a task complexity level and triggers the full routing process.

*Note: A level-N prompt isn't always assigned to the Nth-most capable model due to the classifier's subjective decisions.*

In [17]:
def test_generation_routing(level):
    question = generate_question(level=level)
    answer_model = route_question_to_model(question, ranked_model_names)
    messages = [{"role": "user", "content": question}]

    response =qa_models[answer_model].chat.completions.create(
            model=answer_model,  # or your planner model
            messages=messages
        )
    print(f"Question : {question}")
    print(f"Routed to {answer_model}")
    display(Markdown(response.choices[0].message.content))

In [ ]:
test_generation_routing(level=1)

In [ ]:
test_generation_routing(level=2)

In [ ]:
test_generation_routing(level=3)

In [ ]:
test_generation_routing(level=4)

In [ ]:
test_generation_routing(level=5)